In [2]:
import numpy as np
import polars as pl
import pandas as pd
from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os
import matplotlib.pyplot as plt
import seaborn as sns

import re
from colorama import Fore, Style

from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

SEED = 42
n_splits = 5

In [ ]:
# 各ファイルのパスを表示して確認
for filename in os.listdir('kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'):
    full_path = os.path.join('kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet', filename, 'part-0.parquet')
    print(full_path)


kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0d01bbf2/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=cefdb7fe/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=58391429/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=2ca2206f/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=19455336/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=ca33a5e7/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=92bb8516/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=2812951b/part-0.parquet
kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=6b6467f4/part-0.parquet
kaggle/input/child-mind-institute-pro

In [6]:
import os

# Parquetファイルが本当にParquetか確認する
dirname = 'kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'
for filename in os.listdir(dirname):
    file_path = os.path.join(dirname, filename, 'part-0.parquet')
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as f:
            magic_bytes = f.read(4)
            print(f"File: {file_path}, Magic Bytes: {magic_bytes}")


File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0d01bbf2/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=cefdb7fe/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=58391429/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=2ca2206f/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=19455336/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=ca33a5e7/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=92bb8516/part-0.parquet, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-int

In [7]:
import os

# 各Parquetファイルのサイズを表示
dirname = 'kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'
for filename in os.listdir(dirname):
    file_path = os.path.join(dirname, filename, 'part-0.parquet')
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path)
        print(f"File: {file_path}, Size: {file_size} bytes")


File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0d01bbf2/part-0.parquet, Size: 6571868 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=cefdb7fe/part-0.parquet, Size: 7602176 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=58391429/part-0.parquet, Size: 6573075 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=2ca2206f/part-0.parquet, Size: 7572782 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=19455336/part-0.parquet, Size: 7184455 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=ca33a5e7/part-0.parquet, Size: 7602176 bytes
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=92bb8516/part-0.parquet, Size: 2194086 bytes
File: kaggle/input/child-mind-institute-problematic-internet-u

In [8]:
import os

dirname = 'kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'
for filename in os.listdir(dirname):
    file_path = os.path.join(dirname, filename, 'part-0.parquet')
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path)
        with open(file_path, 'rb') as f:
            magic_bytes = f.read(4)
        print(f"File: {file_path}, Size: {file_size} bytes, Magic Bytes: {magic_bytes}")


File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0d01bbf2/part-0.parquet, Size: 6571868 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=cefdb7fe/part-0.parquet, Size: 7602176 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=58391429/part-0.parquet, Size: 6573075 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=2ca2206f/part-0.parquet, Size: 7572782 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=19455336/part-0.parquet, Size: 7184455 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=ca33a5e7/part-0.parquet, Size: 7602176 bytes, Magic Bytes: b'PAR1'
File: kaggle/input/child-mind-institute-problematic-internet-use

In [4]:
%%time
def process_file(filename, dirname):
    # 指定されたディレクトリとファイル名から Parquet ファイルを読み込み、データフレームに格納
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    # データフレームから 'step' 列を削除
    df.drop('step', axis=1, inplace=True)
    
    # データフレームの基本統計量を計算し、一次元配列に変換
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    # 指定されたディレクトリ内のファイル名を取得
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        # 各ファイルに対して process_file 関数を実行
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
        
        # results リストの各タプルをアンパックし、基本統計量とファイル名の一部をそれぞれ stats と indexes に格納
        stats, indexes = zip(*results)
        
        # stats リストをデータフレームに変換
        df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
        # indexes リストをデータフレームに追加
        df['id'] = indexes
        
        return df
    

os.chdir('/home')
print(os.getcwd())

# データの読み込み
train = pd.read_csv('kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')


train_ts = load_time_series('kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet')
test_ts = load_time_series('kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet')

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove('id')

train = pd.merge(train, train_ts, how="left", on="id")
test = pd.merge(test, test_ts, how="left", on="id")

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

# モデルのトレーニングに使用する特徴量のみを抽出
train = train[featuresCols]
# sii 列の欠損値を含む行を削除
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
        'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    for c in cat_c:
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping_train = create_mapping(col, train)
    mapping_test = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping_train).astype(int)
    test[col] = test[col].replace(mapping_test).astype(int)
    
print(f'Train Shape : {train.shape} || Test Shape : ')

/home


100%|██████████| 2/2 [00:00<00:00, 16.59it/s]


Train Shape : (2736, 155) || Test Shape : 
CPU times: user 2min 17s, sys: 11.9 s, total: 2min 29s
Wall time: 39.3 s
